### 1. Import libraries


In [74]:
import numpy as np
import cv2
from moviepy import editor

In [76]:

def hough_transform(image):
    rho = 1
    theta = np.pi/180
    threshold = 20
    minLineLength = 20
    maxLineGap = 500
    lines = cv2.HoughLinesP(image, rho=rho, theta=theta, threshold=threshold,
                            minLineLength=minLineLength, maxLineGap=maxLineGap)
    return lines if lines is not None else []


In [75]:
def region_selection(image):
	mask = np.zeros_like(image) 
	if len(image.shape) > 2:
		channel_count = image.shape[2]
		ignore_mask_color = (255,) * channel_count
	else:
		ignore_mask_color = 255
	rows, cols = image.shape[:2]
	bottom_left = [cols * 0.1, rows * 0.95]
	top_left	 = [cols * 0.4, rows * 0.6]
	bottom_right = [cols * 0.9, rows * 0.95]
	top_right = [cols * 0.6, rows * 0.6]
	vertices = np.array([[bottom_left, top_left, top_right, bottom_right]], dtype=np.int32)
	cv2.fillPoly(mask, vertices, ignore_mask_color)
	masked_image = cv2.bitwise_and(image, mask)
	return masked_image


In [77]:
def average_slope_intercept(lines):
    left_lines = []
    left_weights = []
    right_lines = []
    right_weights = []
    for line in lines:
        for x1, y1, x2, y2 in line:
            if x1 == x2:
                continue
            slope = (y2 - y1) / (x2 - x1)
            intercept = y1 - (slope * x1)
            length = np.sqrt(((y2 - y1) ** 2) + ((x2 - x1) ** 2))
            if slope < 0:
                left_lines.append((slope, intercept))
                left_weights.append((length))
            else:
                right_lines.append((slope, intercept))
                right_weights.append((length))
    left_lane = np.dot(left_weights, left_lines) / np.sum(left_weights) if len(left_weights) > 0 else None
    right_lane = np.dot(right_weights, right_lines) / np.sum(right_weights) if len(right_weights) > 0 else None
    return left_lane, right_lane


In [78]:
def pixel_points(y1, y2, line):
	if line is None:
		return None
	slope, intercept = line
	x1 = int((y1 - intercept)/slope)
	x2 = int((y2 - intercept)/slope)
	y1 = int(y1)
	y2 = int(y2)
	return ((x1, y1), (x2, y2))

In [79]:
def lane_lines(image, lines):
    if lines is None or len(lines) == 0:
        return None, None
    left_lane, right_lane = average_slope_intercept(lines)
    y1 = image.shape[0]
    y2 = int(y1 * 0.52)  # Adjust the param to get the line length(smaller coeff» longer line)
    left_line = pixel_points(y1, y2, left_lane)
    right_line = pixel_points(y1, y2, right_lane)
    return left_line, right_line

In [80]:
def draw_lane_lines(image, lines, color=[0, 255, 0], thickness=12):
    line_image = np.zeros_like(image)
    if lines:
        for line in lines:
            if line is not None:
                cv2.line(line_image, *line, color, thickness)
    return cv2.addWeighted(image, 1.0, line_image, 1.0, 0.0)

In [81]:
def frame_processor(image):
	grayscale = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
	kernel_size = 3
	blur = cv2.GaussianBlur(grayscale, (kernel_size, kernel_size), 0)
	low_t = 20
	high_t = 100
	edges = cv2.Canny(blur, low_t, high_t)
	region = region_selection(edges)
	hough = hough_transform(region)
	lanes = lane_lines(image, hough)
	result = draw_lane_lines(image, lanes)
	
	grayscale_3ch = cv2.cvtColor(grayscale, cv2.COLOR_GRAY2BGR)
	edges_3ch = cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR)
	region_3ch = cv2.cvtColor(region, cv2.COLOR_GRAY2BGR)
	
	concatenated = np.concatenate((grayscale_3ch, edges_3ch, region_3ch, result), axis=1)
	
	return concatenated


In [82]:
def process_video(test_video, output_video, fps=25.0):
    try:
        input_video = editor.VideoFileClip(test_video, audio=False)
    except Exception as e:
        raise ValueError(f"Error loading video {test_video}: {e}")
    fps = float(input_video.fps) if input_video.fps is not None else fps
    print(f"Processing video {test_video} at {fps} FPS")

    processed = input_video.fl_image(frame_processor)
    processed.write_videofile(output_video, fps=fps, audio=False)

process_video('test2.mp4','output.mp4')

Processing video test2.mp4 at 50.0 FPS
Moviepy - Building video output.mp4.
Moviepy - Writing video output.mp4



Moviepy - Done !
Moviepy - video ready output.mp4
